In [2]:
from functools import reduce
import random
import pandas as pd
import names
from datetime import datetime, timedelta

In [3]:
gate_columns = [
    "Gate ID",
    "Terminal Name",
    "Gate Number",
    "Status",
    "Airport ID"
]

airport_columns = [
    "Airport ID",
    "Name",
    "City"
]

weather_columns = [
    "Weather ID",
    "Conditions",
    "Update Time",
    "Airport ID"
]

aircraft_columns = [
    "Aircraft ID",
    "Aircraft Type",
    "Current Status"
]

passenger_columns = [
    "Passenger ID",
    "Name",
    "Ticket Number",
    "Flight Number"
]

crew_columns = [
    "Crew ID",
    "Crew Type",
    "Member Name",
    "Flight Number"
]

flight_columns = [
    "Flight Number",
    "Departure Time",
    "Arrival Time",
    "Flight Status",
    "Aircraft ID",
    "Departure Gate ID",
    "Departure Airport ID",
    "Arrival Gate ID",
    "Arrival Airport ID"
]


aircraft_types = ["Airbus A320", "Airbus A380", "Boeing 737", "Boeing 747", "Boeing 787"]

weather_conditions = ["Clear skies", "Partly cloudy", "Light rain", "Snow", 
                                "Heavy rain", "Foggy", "Strong wind", "Thunderstorms with rain", 
                                "Thunderstorms with lightning", "Frost", "Icy runway"]



In [4]:
flights = pd.DataFrame(columns=flight_columns)

In [5]:
def generate_random_datetimes(seed=None):
    if seed is not None:
        random.seed(seed)
    
    # Generate a random date within a given range
    start_date = datetime(2020, 1, 1)  # Start date
    end_date = datetime(2023, 12, 31)  # End date
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    
    # First random datetime
    random_date1 = start_date + timedelta(days=random_days)
    random_time1 = timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    datetime1 = datetime.combine(random_date1, (datetime.min + random_time1).time())
    
    # Decide if the second datetime will be on the same day or the next day
    same_day = random.choice([True, False])
    
    if same_day:
        # Second datetime on the same day
        random_time2 = timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
        datetime2 = datetime.combine(random_date1, (datetime.min + random_time2).time())
    else:
        # Second datetime on the next day
        random_date2 = random_date1 + timedelta(days=1)
        random_time2 = timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
        datetime2 = datetime.combine(random_date2, (datetime.min + random_time2).time())
    
    # Ensure the datetime2 is after datetime1 if they are on the same day
    if datetime2 < datetime1 and same_day:
        datetime1, datetime2 = datetime2, datetime1

    return datetime1, datetime2

# Example usage with a seed:
seed_value = 42
dt1, dt2 = generate_random_datetimes(seed=seed_value)
print("Datetime 1:", dt1)
print("Datetime 2:", dt2)

    

Datetime 1: 2023-08-02 03:01:47
Datetime 2: 2023-08-03 07:14:08


In [ ]:
for i in range(50000):
    